In [1]:
print('start ch uploads')
#Clickhouse db w/ Goldsky
# https://clickhouse.com/docs/en/integrations/python

import requests as r
import pandas as pd
import clickhouse_connect as cc
import os

import sys
sys.path.append("../helper_functions")
import duneapi_utils as d
import pandas_utils as p
import clickhouse_utils as ch
import csv_utils as cu
import google_bq_utils as bqu
import opstack_metadata_utils as ops
import goldsky_db_utils as gsb
sys.path.pop()

import time

start ch uploads


In [2]:
client = ch.connect_to_clickhouse_db() #Default is OPLabs DB
# client.close()

table_name = 'daily_aggegate_l2_chain_usage_goldsky'

In [3]:
# bqu.delete_bq_table('api_table_uploads',table_name)

In [4]:
block_time_sec = 2

trailing_days = 14
max_execution_secs = 3000

In [5]:
# chain_mappings_list = [
#     # {'schema_name': 'zora', 'display_name': 'Zora', 'has_blob_fields': True},
#     # {'schema_name': 'pgn', 'display_name': 'Public Goods Network', 'has_blob_fields': True},
#     # {'schema_name': 'base', 'display_name': 'Base', 'has_blob_fields': False},
#     # {'schema_name': 'op', 'display_name': 'OP Mainnet', 'has_blob_fields': True},
#     {'schema_name': 'mode', 'display_name': 'Mode', 'has_blob_fields': True},
#     {'schema_name': 'metal', 'display_name': 'Metal', 'has_blob_fields': True},
#     {'schema_name': 'fraxtal', 'display_name': 'Fraxtal', 'has_blob_fields': True},
#     # {'schema_name': 'bob', 'display_name': 'BOB (Build on Bitcoin)', 'has_blob_fields': False},
#     {'schema_name': 'cyber', 'display_name': 'Cyber', 'has_blob_fields': True},
#     {'schema_name': 'mint', 'display_name': 'Mint', 'has_blob_fields': True},
#     # Add more mappings as needed
# ]
# chain_mappings_dict = {item['schema_name']: item['display_name'] for item in chain_mappings_list}


In [6]:
# Get Chain List
chain_configs = ops.get_op_stack_metadata_by_data_source('oplabs') # OPLabs db
#Filter to Superchain
chain_configs = chain_configs[chain_configs['alignment']=='OP Chain']

chain_configs

,chain_name,display_name,mainnet_chain_id,chain_layer,alignment,blockchain
0,op,OP Mainnet,10.0,L2,OP Chain,op
6,zora,Zora,7777777.0,L2,OP Chain,zora
10,base,Base,8453.0,L2,OP Chain,base
16,mode,Mode,34443.0,L2,OP Chain,mode
23,fraxtal,Fraxtal,252.0,L2,OP Chain,fraxtal
29,metal,Metal,1750.0,L2,OP Chain,metal
30,mint,Mint,185.0,L2,OP Chain,mint
31,cyber,Cyber,7560.0,L2,OP Chain,cyber


In [7]:
sql_directory = "inputs/sql/"

query_names = [
        # Must match the file name in inputs/sql
        "ch_template_alltime_chain_activity"
]

In [8]:
unified_dfs = []

In [9]:
for qn in query_names:
        for index, chain in chain_configs.iterrows():
                chain_schema = chain['blockchain']
                display_name = chain['display_name']
                # Read the SQL query from file
                with open(os.path.join(sql_directory, f"{qn}.sql"), "r") as file:
                        query = file.read()
                print(qn + ' - ' + chain_schema)

                #Pass in Params to the query
                query = query.replace("@chain_db_name@", chain_schema)
                query = query.replace("@trailing_days@", str(trailing_days))
                query = query.replace("@block_time_sec@", str(block_time_sec))
                query = query.replace("@max_execution_secs@", str(max_execution_secs))

                query = gsb.process_goldsky_sql(query)

                # Execute the query
                result_df = client.query_df(query)
        #         # Write to csv
        #         df.to_csv('outputs/chain_data/' + qn + '.csv', index=False)
        #         # print(df.sample(5))
        #         time.sleep(1)
                
                result_df['chain_raw'] = result_df['chain']
                result_df['chain'] = display_name
                unified_dfs.append(result_df)

        write_df = pd.concat(unified_dfs)
        write_df.to_csv('outputs/chain_data/' + qn + '.csv', index=False)
        
        # # # Print the results

ch_template_alltime_chain_activity - op
ch_template_alltime_chain_activity - zora
ch_template_alltime_chain_activity - base
ch_template_alltime_chain_activity - mode
ch_template_alltime_chain_activity - fraxtal
ch_template_alltime_chain_activity - metal
ch_template_alltime_chain_activity - mint
ch_template_alltime_chain_activity - cyber


In [10]:
# write_df.dtypes
write_df['chain_id'] = write_df['chain_id'].astype('int64')

In [11]:
#BQ Upload
time.sleep(1)
bqu.append_and_upsert_df_to_bq_table(write_df, table_name, unique_keys = ['dt','chain','network'])

Start Writing api_table_uploads.daily_aggegate_l2_chain_usage_goldsky_staging
Data loaded successfully to api_table_uploads.daily_aggegate_l2_chain_usage_goldsky_staging


2024-07-18 12:28:52,015 INFO google_bq_utils Append and upsert to api_table_uploads.daily_aggegate_l2_chain_usage_goldsky completed successfully
2024-07-18 12:28:52,325 INFO google_bq_utils Staging table oplabs-tools-data.api_table_uploads.daily_aggegate_l2_chain_usage_goldsky_staging deleted.


In [12]:
# Copy to Dune
print('upload bq to dune')
sql = '''
SELECT *
FROM `api_table_uploads.daily_aggegate_l2_chain_usage_goldsky`
WHERE dt >= DATE_SUB(CURRENT_DATE(), INTERVAL 366 DAY)
'''
bq_df = bqu.run_query_to_df(sql)

dune_table_name = 'ch_template_alltime_chain_activity'
d.write_dune_api_from_pandas(bq_df, dune_table_name,table_description = dune_table_name)

upload bq to dune
uploading ch_template_alltime_chain_activity
Response status code: 200
Response content: b'{"success":true,"table_name":"ch_template_alltime_chain_activity"}'
table at: dune.oplabspbc.dataset_ch_template_alltime_chain_activity
